In [2]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
#None -> 60000, 28x28x1(이미지 가로 * 이미지 세로 * 채널 수)
X = tf.placeholder(dtype=tf.float32, shape=[None, 28, 28, 1])
Y = tf.placeholder(dtype=tf.float32, shape=[None, 10])
#3x3크기, 1개의 채널, 32개의 필터를 가지는 커널.
W1 = tf.Variable(tf.random_normal(shape=[3, 3, 1, 32], stddev=0.01))
#strides의 원소 4개는 [batch, height, width, channels]
L1 = tf.nn.conv2d(X, W1, padding='SAME', strides=[1, 1, 1, 1])
L1 = tf.nn.relu(L1)
#ksize의 가운데 원소 2개는 [height, width]
#strides의 원소 4개는 [batch, height, width, channels] 가로 방향으로 2씩 이동하고, 다 이동을 마치면 다시 왼쪽으로 돌아온 뒤, 아래로 2칸 이동 후 다시 오른쪽으로 2씩 이동.
L1 = tf.nn.max_pool(L1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

W2 = tf.Variable(tf.random_normal(shape=[3, 3, 32, 64], stddev=0.01))
L2 = tf.nn.conv2d(L1, W2, padding='SAME', strides=[1, 1, 1, 1])
L2 = tf.nn.relu(L2)
L2 = tf.nn.max_pool(L2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

W3 = tf.Variable(tf.random_normal(shape=[7*7*64, 256], stddev=0.01))
L3 = tf.reshape(L2, [-1, 7*7*64])
L3 = tf.matmul(L3, W3)
L3 = tf.nn.relu(L3)

W4 = tf.Variable(tf.random_normal([256, 10], stddev=0.01))
model = tf.matmul(L3, W4)

In [4]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=model))
optimizer = tf.train.AdamOptimizer(learning_rate=0.01).minimize(cost)

In [5]:
batch_size = 100
total_batch = int(mnist.train.num_examples / batch_size)

In [6]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(5):
        total_cost = 0
        for i in range(total_batch):
            x_batch, y_batch = mnist.train.next_batch(batch_size)
            #print(x_batch.shape)
            x_batch = x_batch.reshape(-1, 28, 28, 1)
            _, cost_val = sess.run([optimizer, cost], feed_dict={X : x_batch, Y : y_batch})

            total_cost = total_cost + cost_val
        print("epoch : ", epoch+1, "cost : ", total_cost / total_batch)
        
    is_correct = tf.equal(tf.argmax(model, 1), tf.argmax(Y, 1))
    accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
    print('정확도:', sess.run(accuracy, feed_dict={X: mnist.test.images.reshape(-1, 28, 28, 1), Y: mnist.test.labels}))

epoch :  1 cost :  0.14537572401999072
epoch :  2 cost :  0.05066927163396031
epoch :  3 cost :  0.03890611537425271
epoch :  4 cost :  0.03307473963518119
epoch :  5 cost :  0.03198950803310833
정확도: 0.9865
